In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from numpy import dot
from numpy.linalg import norm
import time
import datetime

In [19]:
cvocp_user_course_timing=pd.read_csv('chulamoocs/01_cvocp_user_course_timing_masked.csv')
cvocp_user_test_score=pd.read_csv('chulamoocs/02_cvocp_user_test_score_masked.csv')
cvocp_course=pd.read_csv('chulamoocs/03_cvocp_course_masked.csv')
cv_course_poster=pd.read_csv('chulamoocs/04_courseville_course_poster_masked.csv')
cvocp_user_info=pd.read_csv('chulamoocs/05_cvocp_user_info_masked.csv')
cv_content_video=pd.read_csv('chulamoocs/06_courseville_content_video_masked.csv')


In [20]:
cvocp_user_course_timing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049508 entries, 0 to 1049507
Data columns (total 7 columns):
record_id      1049508 non-null int64
cv_uid         1049508 non-null object
cv_cid         1049508 non-null object
action_type    1049508 non-null object
assoc_id       1049508 non-null object
assoc_id2      1049508 non-null object
timestamp      1049508 non-null int64
dtypes: int64(2), object(5)
memory usage: 56.0+ MB


In [21]:
cvocp_user_test_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375140 entries, 0 to 375139
Data columns (total 6 columns):
user_test_score_id    375140 non-null int64
cv_uid                375140 non-null object
test_nid              375140 non-null object
sub_id                375140 non-null object
point                 375140 non-null int64
total                 375140 non-null int64
dtypes: int64(3), object(3)
memory usage: 17.2+ MB


In [22]:
cvocp_course.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 14 columns):
cv_cid               175 non-null object
course_title         175 non-null object
course_no            175 non-null object
year                 175 non-null int64
semester             175 non-null int64
instructor_string    159 non-null object
course_start         175 non-null int64
course_end           175 non-null int64
reg_start            175 non-null int64
reg_end              175 non-null int64
roster_limit         175 non-null int64
reg_workflow         175 non-null object
course_type          175 non-null object
status               175 non-null int64
dtypes: int64(8), object(6)
memory usage: 19.3+ KB


In [23]:
cv_course_poster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
cv_cid                175 non-null object
course_pretitle       175 non-null object
course_title          174 non-null object
course_description    155 non-null object
dtypes: object(4)
memory usage: 5.6+ KB


In [24]:
cvocp_user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51036 entries, 0 to 51035
Data columns (total 6 columns):
cv_uid         51036 non-null object
education      51035 non-null object
occupation     51035 non-null object
gender         51035 non-null object
yearofbirth    51035 non-null float64
residence      51035 non-null object
dtypes: float64(1), object(5)
memory usage: 2.3+ MB


In [25]:
cv_content_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2683 entries, 0 to 2682
Data columns (total 3 columns):
v_id               2683 non-null object
cv_cid             2683 non-null object
video_source_id    2683 non-null object
dtypes: object(3)
memory usage: 63.0+ KB


In [26]:
cvocp_user_course_timing.head()
# cat feature - action_type
# timestamp -> time diff
# create new table mapping_user_course for mapping user and course 
# recommend courseid to user!? n-courses??

,record_id,cv_uid,cv_cid,action_type,assoc_id,assoc_id2,timestamp
0,1,KP0XGECF1SGVOS2O24VF,UMF4ZAMAJ3UAKSY5URJR,quiz_submitted,JMYCQPQ9ZCPXEEEMSHIH,SRGWN4LOSQEK8A51ENFQ,1530413688
1,2,KP0XGECF1SGVOS2O24VF,UMF4ZAMAJ3UAKSY5URJR,quiz_submitted,JMYCQPQ9ZCPXEEEMSHIH,SRGWN4LOSQEK8A51ENFQ,1530413709
2,3,KP0XGECF1SGVOS2O24VF,UMF4ZAMAJ3UAKSY5URJR,quiz_submitted,JMYCQPQ9ZCPXEEEMSHIH,SRGWN4LOSQEK8A51ENFQ,1530413724
3,4,KP0XGECF1SGVOS2O24VF,UMF4ZAMAJ3UAKSY5URJR,quiz_submitted,JMYCQPQ9ZCPXEEEMSHIH,SRGWN4LOSQEK8A51ENFQ,1530413741
4,5,KH8UA5P183ESRY8BQSNP,7M978UYOQ7J2A0EGDK3D,quiz_opened,83PK3M4EQRNVOPDASXCL,7LSY9RBX6NM0NX4RC4RY,1530413927


In [27]:
cvocp_user_test_score.head()
# cont. var - point total

,user_test_score_id,cv_uid,test_nid,sub_id,point,total
0,1,GEW2S6U24ZMJTSJLEPTK,EVYPCWEHD3NO60SVF2DD,ZW146UB6QZPD643Y4FJ3,2,2
1,2,WCB8KSCK643JQCORSFQ2,R2QG06ZY6NX9FS2HMAS5,0KL8GRMBK2PLB1TZR515,18,20
2,3,IGUC7NX29Z9WC7989OB0,6VZM0HTL86TV99POT26E,PLRTX6EWY6IJ2N0E138W,14,20
3,4,OLDI3GBOISKBC31PFSE8,R2QG06ZY6NX9FS2HMAS5,2R8GFDQPRYK2VNKLSIFW,17,20
4,5,XQJHIUGB1ZY6GX5PYLF6,717D9JTDZT13NQ79YP66,TP0X7W55O4MHUOSQKI3B,1,2


In [40]:
user_test_score=cvocp_user_test_score.copy()
user_test_score['percent']=user_test_score_agg['point']/user_test_score_agg['total']*100
user_test_score_agg=user_test_score[['cv_uid','percent']].agg(['min','max','std','mean'])

In [28]:
cvocp_course.head()
# cat feature year??, semester, reg_workflow, course_type, status
# cont. var - roster limit
# time - course_start, course_end, reg_start, reg_end

,cv_cid,course_title,course_no,year,semester,instructor_string,course_start,course_end,reg_start,reg_end,roster_limit,reg_workflow,course_type,status
0,USIS5STDNOBQGGHXDDG7,การควบคุมยาสูบ รุ่นที่ 4,TRC.2016,2016,1,NaN,0,0,0,0,45,invited_selfreg,selfpace,1
1,ODRCRDFROFTNYRC0PQU2,Survival Thai (2016),CU.MOOC.0001,2016,2,NaN,1483491600,1496249999,1483491600,1485881999,14,invited_selfreg,selfpace,1
2,9W16HUAN3F0LB16BN1QC,การควบคุมยาสูบ รุ่นที่ 5,TRC.ELearning,2017,1,NaN,1501779600,1528736399,0,0,50,purchase_to_enroll,weekly,1
3,NTFBS3C5O5HPWTX9B8YJ,ภาษาอาหรับในชีวิตประจำวัน,CU.MOOC.0002,2017,1,อ. ดร.ทรงศักดิ์ หมัดสะและ,1504803600,1509469199,1504803600,1508086799,100,purchase_to_enroll,selfpace,1
4,P074WP7NLCCMPJ200PE8,การทำความเข้าใจงบการเงิน,CU.MOOC.0003,2017,1,ผศ. ดร.พิมพ์พนา ปีตธวัชชัย,1504803600,1509469199,1504803600,1508086799,2000,free_selfreg,selfpace,1


In [29]:
cv_course_poster.head()
# text - course_description

,cv_cid,course_pretitle,course_title,course_description
0,USIS5STDNOBQGGHXDDG7,TRC.2016 (2016/1),การควบคุมยาสูบ รุ่นที่ 4,NaN
1,ODRCRDFROFTNYRC0PQU2,CU.MOOC.0001 (2016/2),Survival Thai,<p>This <strong>Survival Thai</strong> course ...
2,9W16HUAN3F0LB16BN1QC,TRC.ELearning (2017/1),การควบคุมยาสูบ รุ่นที่ 5,NaN
3,NTFBS3C5O5HPWTX9B8YJ,CU.MOOC.0002 (2017/1),ภาษาอาหรับในชีวิตประจำวัน,รายวิชาภาษาอาหรับในชีวิตประจำวัน จะเป็นการศึกษ...
4,P074WP7NLCCMPJ200PE8,CU.MOOC.0003 (2017/1),การทำความเข้าใจงบการเงิน,รายวิชาการทำความเข้าใจงบการเงิน จะเป็นการศึกษา...


In [30]:
cvocp_user_info.head()
# cat feature - education, occupation, gender, residence
# yearofbirth -> age 

,cv_uid,education,occupation,gender,yearofbirth,residence
0,KH8UA5P183ESRY8BQSNP,Above Bachelor Degree,Govenment/State Enterprise Employee,Male,1976.0,Bangkok
1,TFH3B91W13YZ8ZUJJC38,Above Bachelor Degree,Other,Male,1987.0,Bangkok
2,D3QMUHBVM4YI6MWUAPZL,Bachelor Degree,Other,Male,1989.0,Bangkok
3,7MG8V8F2J9AF2Y0PS2TZ,Above Bachelor Degree,Private Sector Employee,Male,1989.0,Bangkok
4,S4MFNN7NXR87BMQHYTN9,Above Bachelor Degree,Govenment/State Enterprise Employee,Male,1964.0,Bangkok


In [31]:
cv_content_video.head()
# id/link - video_source_id

,v_id,cv_cid,video_source_id
0,ZIDHTXVVXSQZX0ZH49CN,USIS5STDNOBQGGHXDDG7,VB79884A9OC2KYK6H3WB
1,0UYCENV1YISTZ2YP3MB8,USIS5STDNOBQGGHXDDG7,VB79884A9OC2KYK6H3WB
2,2MBAIA1WVYH7W6N91U9O,USIS5STDNOBQGGHXDDG7,VB79884A9OC2KYK6H3WB
3,MCTF3W40S5TKQD9QA8W9,USIS5STDNOBQGGHXDDG7,VB79884A9OC2KYK6H3WB
4,A2C3H4LCS2QEGJUIBNSP,USIS5STDNOBQGGHXDDG7,VB79884A9OC2KYK6H3WB


In [32]:
datetime.datetime.now().year

2020

## Pre-processing data

In [33]:
# User detail

cat_user_info=['education','occupation','gender','residence']
cont_user_info=['yearofbirth']

# One hot encoding on categorical feature of user info data
user_info_dummies=pd.get_dummies(cvocp_user_info,columns=cat_user_info)

# convert yearofbirth to age
current_year=datetime.datetime.now().year
user_info_dummies['age']=user_info_dummies['yearofbirth'].apply(lambda x:current_year-x if x!=np.nan else null )

user_info_dummies=user_info_dummies.drop(['yearofbirth'],axis=1)

In [ ]:
# User testing score


In [34]:
# get course of user
user_course=cvocp_user_course_timing[['cv_uid','cv_cid','timestamp']].drop_duplicates().sort_values(by='timestamp',ascending=True)

In [35]:
# course detail

In [36]:
def get_sim_order(mat_old,mat_new):
  cosine_sim=dot(mat_old, mat_new)/(norm(mat_old)*norm(mat_new))
  return np.argsort(cosine_sim, axis=0)
  #return list of index of similarity score from max to min

In [37]:
def map_index():
  pass

In [38]:
def eval_by_rank(groud_truth,pred):
    pass